# Jupyter Distributed Extension Demo

This notebook demonstrates how to use the jupyter_distributed extension for interactive distributed PyTorch training.

## 1. Load the Extension

In [1]:
%load_ext jupyter_distributed

In [2]:
%dist_debug

=== Distributed Debug Information ===
Process manager exists: False
Communication manager exists: False
Number of ranks: 0


## 2. Initialize Distributed Workers

Start 2 worker processes for distributed training:

In [4]:
%dist_init --num-ranks 2 --gpu-ids 1,2

Distributed workers already running. Use %dist_shutdown to stop them first.


In [5]:
%dist_debug

=== Distributed Debug Information ===
Process manager exists: True
Communication manager exists: True
Number of ranks: 2
Process manager is_running(): True
Number of processes tracked: 2
  Process 0 (PID: 3650458): Running
  Process 1 (PID: 3650459): Running


In [10]:
%dist_reset

=== DISTRIBUTED ENVIRONMENT RESET ===
Performing nuclear shutdown...
All state cleared
You can now run %dist_init to start fresh


In [11]:
%dist_debug

=== Distributed Debug Information ===
Process manager exists: False
Communication manager exists: False
Number of ranks: 0


In [7]:
%dist_reset --nuclear

=== DISTRIBUTED ENVIRONMENT RESET ===
Performing nuclear reset...
All state cleared
You can now run %dist_init to start fresh


In [8]:
%dist_debug

=== Distributed Debug Information ===
Process manager exists: False
Communication manager exists: False
Number of ranks: 0


## 3. Check Worker Status

In [5]:
%dist_status

Distributed cluster status (2 ranks):
Rank 0: ✓ PID 3648619
  ├─ GPU: 1 (NVIDIA GeForce RTX 4090)
  ├─ Memory: 0.0GB / 23.5GB (0.0% used)
  └─ Status: Running

Rank 1: ✓ PID 3648620
  ├─ GPU: 2 (NVIDIA GeForce RTX 4090)
  ├─ Memory: 0.0GB / 23.5GB (0.0% used)
  └─ Status: Running



## 4. Execute Code on All Ranks

The `%%distributed` magic runs code on all worker processes:

In [6]:
%%distributed
import torch
import torch.distributed as dist

print(f"Hello from rank {rank}/{world_size-1}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


=== All ranks ===

--- Rank 0 ---
Hello from rank 0/1
PyTorch version: 2.4.0+cu121
CUDA available: True


--- Rank 1 ---
Hello from rank 1/1
PyTorch version: 2.4.0+cu121
CUDA available: True



## 5. Create and Distribute Tensors

In [7]:
%%distributed
# Create a tensor on each rank
tensor = torch.tensor([0+device, 1+device, 2+device])
tensor = tensor.to("cuda")
print(f"Rank {rank} tensor:\n{tensor}")


=== All ranks ===

--- Rank 0 ---
Rank 0 tensor:
tensor([1, 2, 3], device='cuda:1')


--- Rank 1 ---
Rank 1 tensor:
tensor([2, 3, 4], device='cuda:2')



## 6. Collective Operations

Perform all-reduce to sum tensors across all ranks:

In [8]:
%%distributed
# All-reduce: sum tensors across all ranks
dist.all_reduce(tensor, op=dist.ReduceOp.SUM)
print(f"Rank {rank} after all-reduce:\n{tensor}")


=== All ranks ===

--- Rank 0 ---
Rank 0 after all-reduce:
tensor([3, 5, 7], device='cuda:1')


--- Rank 1 ---
Rank 1 after all-reduce:
tensor([3, 5, 7], device='cuda:2')



## 7. Execute Code on Specific Ranks

Use `%%rank [...]` to run code only on specific ranks:

In [9]:
%%rank [0]
print("This only runs on rank 0 (master rank)")
master_data = torch.tensor([1, 2, 3, 4, 5])
print(f"Master data: {master_data}")


=== Ranks [0] ===

--- Rank 0 ---
This only runs on rank 0 (master rank)
Master data: tensor([1, 2, 3, 4, 5])



In [10]:
%%rank [1]
print("This only runs on rank 1")
worker_data = torch.tensor([10, 20, 30, 40, 50])
print(f"Worker data: {worker_data}")


=== Ranks [1] ===

--- Rank 1 ---
This only runs on rank 1
Worker data: tensor([10, 20, 30, 40, 50])



## 8. Synchronization

Use `%sync` to synchronize all ranks before continuing:

In [11]:
%sync

✓ Synchronized 2 ranks


## 9. Broadcast Operations

In [12]:
%%distributed
if rank == 0:
    broadcast_tensor = torch.tensor([1,2,3], dtype=torch.float32).to(device)
else:
    broadcast_tensor = torch.zeros(3, dtype=torch.float32).to(device)

print(f"Rank {rank} before broadcast: {broadcast_tensor}")

dist.barrier()

dist.broadcast(broadcast_tensor, src=0)

dist.barrier()

print(f"Rank {rank} after broadcast: {broadcast_tensor}")


=== All ranks ===

--- Rank 0 ---
Rank 0 before broadcast: tensor([1., 2., 3.], device='cuda:1')
Rank 0 after broadcast: tensor([1., 2., 3.], device='cuda:1')


--- Rank 1 ---
Rank 1 before broadcast: tensor([0., 0., 0.], device='cuda:2')
Rank 1 after broadcast: tensor([1., 2., 3.], device='cuda:2')



## 10. Error Handling

The extension handles errors gracefully and shows which rank failed:

In [13]:
%%distributed
if rank == 1:
    # This will cause an error on rank 1
    undefined_variable
else:
    print(f"Rank {rank} executes successfully")


=== All ranks ===

--- Rank 0 ---
Rank 0 executes successfully


--- Rank 1 ---
❌ Error: name 'undefined_variable' is not defined
Traceback (most recent call last):
  File "/home/zach/distributed-learning-zoo/jupyter_distributed/src/jupyter_distributed/worker.py", line 145, in _execute_code
    exec(code, self.namespace)
  File "<string>", line 3, in <module>
NameError: name 'undefined_variable' is not defined



## 11. Complex Rank Specifications

You can specify complex rank patterns:

In [14]:
# Initialize with more ranks for this demo
%dist_shutdown --force

Force shutting down distributed workers...
Force shutdown completed
Distributed workers shutdown


In [15]:
%dist_debug

=== Distributed Debug Information ===
Process manager exists: True
Communication manager exists: True
Number of ranks: 2
Process manager is_running(): True
Number of processes tracked: 2
  Process 0 (PID: 3648619): Running
  Process 1 (PID: 3648620): Running


In [16]:
%dist_init --num-ranks 4

Distributed workers already running. Use %dist_shutdown to stop them first.


In [18]:
%dist_status

Distributed cluster status (2 ranks):
Rank 0: ✓ PID 3646622
  ├─ GPU: 1 (NVIDIA GeForce RTX 4090)
  ├─ Memory: 0.0GB / 23.5GB (0.0% used)
  ├─ Reserved: 0.0GB
  └─ Status: Running

Rank 1: ✓ PID 3646623
  ├─ GPU: 2 (NVIDIA GeForce RTX 4090)
  ├─ Memory: 0.0GB / 23.5GB (0.0% used)
  ├─ Reserved: 0.0GB
  └─ Status: Running



In [55]:
%%rank [0,2]

from IPython import get_ipython
ip = get_ipython()
magic = ip.magics_manager.magics['line']['dist_status']
magic(ip, '')

print(f"This runs on even ranks: {rank}")



=== Ranks [0] ===

--- Rank 0 ---
❌ Error: 'NoneType' object has no attribute 'magics_manager'
Traceback (most recent call last):
  File "/home/zach/distributed-learning-zoo/jupyter_distributed/src/jupyter_distributed/worker.py", line 145, in _execute_code
    exec(code, self.namespace)
  File "<string>", line 4, in <module>
AttributeError: 'NoneType' object has no attribute 'magics_manager'



In [ ]:
%%rank[1-3]
print(f"This runs on ranks 1 through 3: {rank}")

## 12. Cleanup

Always shutdown the distributed workers when done:

In [ ]:
%dist_shutdown

## Summary

The jupyter_distributed extension provides:

- **%dist_init**: Initialize distributed workers
- **%dist_status**: Check worker status
- **%dist_shutdown**: Shutdown workers
- **%%distributed**: Execute code on all ranks
- **%%rank[...]**: Execute code on specific ranks
- **%sync**: Synchronize all ranks

This enables interactive experimentation with distributed PyTorch algorithms directly in Jupyter notebooks!